In [1]:
import pandas as pd

In [2]:
df = []
cvp = pd.read_csv(r"C:\THI_workspace\6731\信令資料\CVP資料\CVP資料\桃園整規CVP旅次資料\20210331旅次數_上午.csv")
df.append(cvp)
cvp = pd.read_csv(r"C:\THI_workspace\6731\信令資料\CVP資料\CVP資料\桃園整規CVP旅次資料\20210401旅次數_上午.csv")
df.append(cvp)
cvp = pd.concat(df)

xy = pd.read_excel(r"C:\Users\kjchang\Downloads\15附錄四、交通分區表 (2)-converted (1).xlsx", sheet_name= 'GIS_縣市鄉鎮市區')
xy = xy[['ID','X', 'Y']].drop_duplicates()
cvp = pd.merge(pd.merge(cvp, xy , left_on = 'o', right_on='ID'),xy , left_on = 'd', right_on='ID', suffixes=['_o','_d'])

In [3]:
o_col = 'o'
d_col = 'd'
o_x_col = 'X_o'
o_y_col = 'Y_o'
d_x_col = 'X_d'
d_y_col = 'Y_d'
count_col = 'trip'
date_col = 'date'

# countdf = cvp.groupby([o_col, d_col]).agg({o_x_col:'mean', o_y_col:'mean',  d_x_col:'mean', d_y_col:'mean', count_col : 'sum', date_col:'nunique'}).reset_index()
# countdf[count_col] = countdf[count_col] / countdf[date_col]
# countdf = countdf.drop(columns = date_col).sort_values(count_col,ascending=False).reset_index(drop = True)

In [5]:
# # 合併 o_col 和 d_col，並取得唯一值與排序
# place = pd.concat([cvp[[o_col]].rename(columns={o_col: 'Place'}),
#                    cvp[[d_col]].rename(columns={d_col: 'Place'})]) \
#            .drop_duplicates() \
#            .sort_values('Place') \
#            .reset_index(drop=True).reset_index(names='PlaceID')

In [7]:
# 合併 o_col 和 d_col，並取得唯一值與排序
place = pd.concat([cvp[[o_col, o_x_col, o_y_col]].rename(columns={o_col: 'PlaceName', o_x_col:'PlaceLng', o_y_col:'PlaceLat'}),
                   cvp[[d_col, d_x_col, d_y_col]].rename(columns={d_col: 'PlaceName', d_x_col:'PlaceLng', d_y_col:'PlaceLat'})]) \
           .drop_duplicates(subset=['PlaceName']) \
           .sort_values('PlaceName') \
           .reset_index(drop=True).reset_index(names='PlaceID')
place

,PlaceID,PlaceName,PlaceLng,PlaceLat
0,0,1,121.54569,25.1380
1,1,2,121.56457,25.1074
2,2,3,121.54779,25.0978
3,3,4,121.55022,25.1014
4,4,5,121.53745,25.1108
...,...,...,...,...
1136,1136,1140,121.09960,24.7506
1137,1137,1141,121.09334,24.5283
1138,1138,1142,121.08297,24.7456
1139,1139,1143,121.13182,24.7217


In [28]:
import numpy as np
df_copy = cvp.copy()
df_copy = pd.merge(df_copy, place.rename(columns = {'PlaceName':o_col}), on = o_col)
df_copy = pd.merge(df_copy, place.rename(columns = {'PlaceName':d_col}), on = d_col, suffixes=['_o','_d'])
df_copy['OD'] = df_copy['PlaceID_o'].astype(str) + '-' + df_copy['PlaceID_d'].astype(str)
df_copy
# 確保小的 PlaceID 放前面，大的放後面
df_copy['Pair'] = np.where(df_copy['PlaceID_o'] < df_copy['PlaceID_d'],
                     df_copy['PlaceID_o'].astype(str) + '-' + df_copy['PlaceID_d'].astype(str),
                     df_copy['PlaceID_d'].astype(str) + '-' + df_copy['PlaceID_o'].astype(str))
# df_copy.groupby(['Pair']).agg({o_x_col:'mean', o_y_col:'mean',  d_x_col:'mean', d_y_col:'mean', count_col : 'sum', date_col:'nunique'}).reset_index()

In [29]:
place = pd.concat([cvp[[o_col, o_x_col, o_y_col]].rename(columns={o_col: 'PlaceName', o_x_col:'PlaceLng', o_y_col:'PlaceLat'}),
                   cvp[[d_col, d_x_col, d_y_col]].rename(columns={d_col: 'PlaceName', d_x_col:'PlaceLng', d_y_col:'PlaceLat'})]) \
           .drop_duplicates(subset=['PlaceName']) \
           .sort_values('PlaceName') \
           .reset_index(drop=True).reset_index(names='PlaceID')
df_copy = cvp.copy()
df_copy = pd.merge(df_copy, place.rename(columns = {'PlaceName':o_col}), on = o_col)
df_copy = pd.merge(df_copy, place.rename(columns = {'PlaceName':d_col}), on = d_col, suffixes=['_o','_d'])
df_copy['OD'] = df_copy['PlaceID_o'].astype(str) + '-' + df_copy['PlaceID_d'].astype(str)
df_copy
# 確保小的 PlaceID 放前面，大的放後面
df_copy['Pair'] = np.where(df_copy['PlaceID_o'] < df_copy['PlaceID_d'],
                     df_copy['PlaceID_o'].astype(str) + '-' + df_copy['PlaceID_d'].astype(str),
                     df_copy['PlaceID_d'].astype(str) + '-' + df_copy['PlaceID_o'].astype(str))

In [31]:
countdf = df_copy.groupby(['Pair'], as_index=False).agg({count_col: 'sum',date_col:'nunique'})
countdf[count_col] = countdf[count_col] / countdf[date_col]
countdf = countdf.drop(columns = date_col).sort_values(count_col,ascending=False).reset_index(drop = True)
countdf[['PlaceID1', 'PlaceID2']] = countdf['Pair'].str.split('-', expand=True).astype(int)
countdf
# # 準備 place 資料表的 PlaceID 對應座標，只處理一次
# place_rename = place.rename(columns={'PlaceID': 'PlaceID1'})
# place_rename2 = place.rename(columns={'PlaceID': 'PlaceID2'})
# # 合併 PlaceID1 和 PlaceID2 的對應經緯度座標
# countdf = countdf.merge(place_rename, on='PlaceID1')\
#                  .merge(place_rename2, on='PlaceID2', suffixes=['_o', '_d'])
# # 篩選所需欄位並重新命名
# countdf = countdf[['Pair', count_col, 'PlaceLng_o', 'PlaceLat_o', 'PlaceLng_d', 'PlaceLat_d']]\
#            .rename(columns={'PlaceLng_o': o_x_col, 
#                             'PlaceLat_o': o_y_col, 
#                             'PlaceLng_d': d_x_col, 
#                             'PlaceLat_d': d_y_col})

,Pair,trip,PlaceID1,PlaceID2
0,909-1127,1184.0,909,1127
1,821-1128,587.5,821,1128
2,822-1128,552.0,822,1128
3,913-1127,546.0,913,1127
4,903-1127,515.0,903,1127
...,...,...,...,...
218563,479-608,1.0,479,608
218564,479-607,1.0,479,607
218565,479-606,1.0,479,606
218566,479-605,1.0,479,605


In [19]:
countdf

,Pair,trip,X_o,Y_o,X_d,Y_d
0,909-1127,1184.0,121.31032,24.9260,121.28919,25.0378
1,821-1128,587.5,121.27127,24.9693,121.28773,25.0413
2,822-1128,552.0,121.27032,24.9533,121.28773,25.0413
3,913-1127,546.0,121.28484,24.9002,121.28919,25.0378
4,903-1127,515.0,121.07294,24.9101,121.28919,25.0378
...,...,...,...,...,...,...
218563,479-608,1.0,121.51786,24.9681,121.29374,24.9880
218564,479-607,1.0,121.51786,24.9681,121.30057,24.9905
218565,479-606,1.0,121.51786,24.9681,121.30098,24.9880
218566,479-605,1.0,121.51786,24.9681,121.32130,24.9774


In [11]:
# # Groupby 並計算總和
# countdf = df_copy.groupby(['Pair'], as_index=False).agg({count_col: 'sum'})
# # 拆分 'Pair' 欄位為 PlaceID1 和 PlaceID2 並轉為 int 類型
# countdf[['PlaceID1', 'PlaceID2']] = countdf['Pair'].str.split('-', expand=True).astype(int)
# # 準備 place 資料表的 PlaceID 對應座標，只處理一次
# place_rename = place.rename(columns={'PlaceID': 'PlaceID1'})
# place_rename2 = place.rename(columns={'PlaceID': 'PlaceID2'})
# # 合併 PlaceID1 和 PlaceID2 的對應經緯度座標
# countdf = countdf.merge(place_rename, on='PlaceID1')\
#                  .merge(place_rename2, on='PlaceID2', suffixes=['_o', '_d'])
# # 篩選所需欄位並重新命名
# countdf = countdf[['Pair', count_col, 'PlaceLng_o', 'PlaceLat_o', 'PlaceLng_d', 'PlaceLat_d']]\
#            .rename(columns={'PlaceLng_o': o_x_col, 
#                             'PlaceLat_o': o_y_col, 
#                             'PlaceLng_d': d_x_col, 
#                             'PlaceLat_d': d_y_col})

In [20]:
# 產生OD帶寬
def get_OD_line_shp(df, o_col, d_col, o_x_col, o_y_col, d_x_col, d_y_col, count_col, date_col, how = 'countd' ,combine = True):
    '''
    Parameters:
    df (dataframe) : 要計算的表格，例如信令資料、票證資料統計圖表
    o_col (str) : 起點的名稱 / 站序 / 可判別的欄位
    d_col (str) : 迄點的名稱 / 站序 / 可判別的欄位
    o_x_col (str) : 起點的經度 (wgs84)
    o_y_col (str) : 起點的緯度 (wgs84)
    d_x_col (str) : 迄點的經度 (wgs84)
    d_y_col (str) : 迄點的緯度 (wgs84)
    count_col (str) : 需要統計依據的量值
    combine (Boolean) : True為去定義是否要把同一對起訖端點的量合併成同一條統計量，False則僅顯示OD
    how (str) : 填入'sum'或'mean'
    OthersObject:
    countdf (dataframe):統計出來的OD表 (尚未轉成geodataframe)
    '''
    import numpy as np
    import pandas as pd

    # 1. 不合併雙向OD
    if combine == False : 
        if how != 'countd':
            countdf = df.groupby([o_col, d_col]).agg({o_x_col:'mean', o_y_col:'mean',  d_x_col:'mean', d_y_col:'mean', count_col : how}).reset_index().sort_values(count_col,ascending=False).reset_index(drop = True)
        elif how == 'countd':
            countdf = df.groupby([o_col, d_col]).agg({o_x_col:'mean', o_y_col:'mean',  d_x_col:'mean', d_y_col:'mean', count_col : 'sum', date_col:'nunique'}).reset_index()
            countdf[count_col] = countdf[count_col] / countdf[date_col]
            countdf = countdf.drop(columns = date_col).sort_values(count_col,ascending=False).reset_index(drop = True)
    else :
        place = pd.concat([cvp[[o_col, o_x_col, o_y_col]].rename(columns={o_col: 'PlaceName', o_x_col:'PlaceLng', o_y_col:'PlaceLat'}),
                        cvp[[d_col, d_x_col, d_y_col]].rename(columns={d_col: 'PlaceName', d_x_col:'PlaceLng', d_y_col:'PlaceLat'})]) \
                .drop_duplicates(subset=['PlaceName']) \
                .sort_values('PlaceName') \
                .reset_index(drop=True).reset_index(names='PlaceID')
        df_copy = cvp.copy()
        df_copy = pd.merge(df_copy, place.rename(columns = {'PlaceName':o_col}), on = o_col)
        df_copy = pd.merge(df_copy, place.rename(columns = {'PlaceName':d_col}), on = d_col, suffixes=['_o','_d'])
        df_copy['OD'] = df_copy['PlaceID_o'].astype(str) + '-' + df_copy['PlaceID_d'].astype(str)
        df_copy
        # 確保小的 PlaceID 放前面，大的放後面
        df_copy['Pair'] = np.where(df_copy['PlaceID_o'] < df_copy['PlaceID_d'],
                            df_copy['PlaceID_o'].astype(str) + '-' + df_copy['PlaceID_d'].astype(str),
                            df_copy['PlaceID_d'].astype(str) + '-' + df_copy['PlaceID_o'].astype(str))
        if how != 'countd':
            countdf = df_copy.groupby(['Pair'], as_index=False).agg({count_col: 'sum'}).sort_values(count_col,ascending=False).reset_index(drop = True) # Groupby 並計算總和
            countdf[['PlaceID1', 'PlaceID2']] = countdf['Pair'].str.split('-', expand=True).astype(int) # 拆分 'Pair' 欄位為 PlaceID1 和 PlaceID2 並轉為 int 類型
            
            
            # 準備 place 資料表的 PlaceID 對應座標，只處理一次
            place_rename = place.rename(columns={'PlaceID': 'PlaceID1'})
            place_rename2 = place.rename(columns={'PlaceID': 'PlaceID2'})

            # 合併 PlaceID1 和 PlaceID2 的對應經緯度座標
            countdf = countdf.merge(place_rename, on='PlaceID1')\
                            .merge(place_rename2, on='PlaceID2', suffixes=['_o', '_d'])
            
            # 篩選所需欄位並重新命名
            countdf = countdf[['Pair', count_col, 'PlaceLng_o', 'PlaceLat_o', 'PlaceLng_d', 'PlaceLat_d']]\
                    .rename(columns={'PlaceLng_o': o_x_col, 
                                        'PlaceLat_o': o_y_col, 
                                        'PlaceLng_d': d_x_col, 
                                        'PlaceLat_d': d_y_col})
        elif how == 'countd':
            countdf = df_copy.groupby(['Pair'], as_index=False).agg({count_col: 'sum',date_col:'nunique'})
            countdf[count_col] = countdf[count_col] / countdf[date_col]
            countdf = countdf.drop(columns = date_col).sort_values(count_col,ascending=False).reset_index(drop = True)
            # 準備 place 資料表的 PlaceID 對應座標，只處理一次
            place_rename = place.rename(columns={'PlaceID': 'PlaceID1'})
            place_rename2 = place.rename(columns={'PlaceID': 'PlaceID2'})
            # 合併 PlaceID1 和 PlaceID2 的對應經緯度座標
            countdf = countdf.merge(place_rename, on='PlaceID1')\
                            .merge(place_rename2, on='PlaceID2', suffixes=['_o', '_d'])
            # 篩選所需欄位並重新命名
            countdf = countdf[['Pair', count_col, 'PlaceLng_o', 'PlaceLat_o', 'PlaceLng_d', 'PlaceLat_d']]\
                    .rename(columns={'PlaceLng_o': o_x_col, 
                                        'PlaceLat_o': o_y_col, 
                                        'PlaceLng_d': d_x_col, 
                                        'PlaceLat_d': d_y_col})
    return countdf

In [ ]:
o_col = 'o'
d_col = 'd'
o_x_col = 'X_o'
o_y_col = 'Y_o'
d_x_col = 'X_d'
d_y_col = 'Y_d'
count_col = 'trip'
date_col = 'date'

In [36]:
# 產生OD帶寬
def get_OD_line_shp(df, o_col, d_col, o_x_col, o_y_col, d_x_col, d_y_col, count_col, date_col, how = 'countd' ,combine = True):
    '''
    Parameters:
    df (dataframe) : 要計算的表格，例如信令資料、票證資料統計圖表
    o_col (str) : 起點的名稱 / 站序 / 可判別的欄位
    d_col (str) : 迄點的名稱 / 站序 / 可判別的欄位
    o_x_col (str) : 起點的經度 (wgs84)
    o_y_col (str) : 起點的緯度 (wgs84)
    d_x_col (str) : 迄點的經度 (wgs84)
    d_y_col (str) : 迄點的緯度 (wgs84)
    count_col (str) : 需要統計依據的量值
    combine (Boolean) : True為去定義是否要把同一對起訖端點的量合併成同一條統計量，False則僅顯示OD
    how (str) : 填入'sum'或'mean'
    OthersObject:
    countdf (dataframe):統計出來的OD表 (尚未轉成geodataframe)
    '''
    import numpy as np
    import pandas as pd

    # 1. 不合併雙向OD
    if combine == False : 
        if how != 'countd':
            countdf = df.groupby([o_col, d_col]).agg({o_x_col:'mean', o_y_col:'mean',  d_x_col:'mean', d_y_col:'mean', count_col : how}).reset_index().sort_values(count_col,ascending=False).reset_index(drop = True)
        elif how == 'countd':
            countdf = df.groupby([o_col, d_col]).agg({o_x_col:'mean', o_y_col:'mean',  d_x_col:'mean', d_y_col:'mean', count_col : 'sum', date_col:'nunique'}).reset_index()
            countdf[count_col] = countdf[count_col] / countdf[date_col]
            countdf = countdf.drop(columns = date_col).sort_values(count_col,ascending=False).reset_index(drop = True)
    else :
        place = pd.concat([cvp[[o_col, o_x_col, o_y_col]].rename(columns={o_col: 'PlaceName', o_x_col:'PlaceLng', o_y_col:'PlaceLat'}),
                        cvp[[d_col, d_x_col, d_y_col]].rename(columns={d_col: 'PlaceName', d_x_col:'PlaceLng', d_y_col:'PlaceLat'})]) \
                .drop_duplicates(subset=['PlaceName']) \
                .sort_values('PlaceName') \
                .reset_index(drop=True).reset_index(names='PlaceID')
        df_copy = cvp.copy()
        df_copy = pd.merge(df_copy, place.rename(columns = {'PlaceName':o_col}), on = o_col)
        df_copy = pd.merge(df_copy, place.rename(columns = {'PlaceName':d_col}), on = d_col, suffixes=['_o','_d'])
        df_copy['OD'] = df_copy['PlaceID_o'].astype(str) + '-' + df_copy['PlaceID_d'].astype(str)
        df_copy
        # 確保小的 PlaceID 放前面，大的放後面
        df_copy['Pair'] = np.where(df_copy['PlaceID_o'] < df_copy['PlaceID_d'],
                            df_copy['PlaceID_o'].astype(str) + '-' + df_copy['PlaceID_d'].astype(str),
                            df_copy['PlaceID_d'].astype(str) + '-' + df_copy['PlaceID_o'].astype(str))
        if how != 'countd':
            countdf = df_copy.groupby(['Pair'], as_index=False).agg({count_col: 'sum'}).sort_values(count_col,ascending=False).reset_index(drop = True) # Groupby 並計算總和
            countdf[['PlaceID1', 'PlaceID2']] = countdf['Pair'].str.split('-', expand=True).astype(int) # 拆分 'Pair' 欄位為 PlaceID1 和 PlaceID2 並轉為 int 類型
            
            # 準備 place 資料表的 PlaceID 對應座標，只處理一次
            place_rename = place.rename(columns={'PlaceID': 'PlaceID1'})
            place_rename2 = place.rename(columns={'PlaceID': 'PlaceID2'})

            # 合併 PlaceID1 和 PlaceID2 的對應經緯度座標
            countdf = countdf.merge(place_rename, on='PlaceID1')\
                            .merge(place_rename2, on='PlaceID2', suffixes=['_o', '_d'])
            
            # 篩選所需欄位並重新命名
            countdf = countdf[['Pair', count_col, 'PlaceLng_o', 'PlaceLat_o', 'PlaceLng_d', 'PlaceLat_d']]\
                    .rename(columns={'PlaceLng_o': o_x_col, 
                                        'PlaceLat_o': o_y_col, 
                                        'PlaceLng_d': d_x_col, 
                                        'PlaceLat_d': d_y_col})
        elif how == 'countd':
            
            countdf = df_copy.groupby(['Pair'], as_index=False).agg({count_col: 'sum',date_col:'nunique'})

            countdf[count_col] = countdf[count_col] / countdf[date_col]
            
            countdf = countdf.drop(columns = date_col).sort_values(count_col,ascending=False).reset_index(drop = True)
            # 準備 place 資料表的 PlaceID 對應座標，只處理一次
            countdf[['PlaceID1', 'PlaceID2']] = countdf['Pair'].str.split('-', expand=True).astype(int)

            place_rename = place.rename(columns={'PlaceID': 'PlaceID1'})
            place_rename2 = place.rename(columns={'PlaceID': 'PlaceID2'})
            # 合併 PlaceID1 和 PlaceID2 的對應經緯度座標
            countdf = countdf.merge(place_rename, on='PlaceID1')\
                            .merge(place_rename2, on='PlaceID2', suffixes=['_o', '_d'])
            # 篩選所需欄位並重新命名
            countdf = countdf[['Pair', count_col, 'PlaceLng_o', 'PlaceLat_o', 'PlaceLng_d', 'PlaceLat_d']]\
                    .rename(columns={'PlaceLng_o': o_x_col, 
                                        'PlaceLat_o': o_y_col, 
                                        'PlaceLng_d': d_x_col, 
                                        'PlaceLat_d': d_y_col})
    return countdf

# 呼叫函式
get_OD_line_shp(cvp, o_col =o_col , d_col = d_col , o_x_col = o_x_col, o_y_col = o_y_col , d_x_col = d_x_col , d_y_col = d_y_col , count_col = count_col, date_col = date_col, how = 'countd' ,combine = False )

,Pair,trip,X_o,Y_o,X_d,Y_d
0,909-1127,1184.0,121.31032,24.9260,121.28919,25.0378
1,821-1128,587.5,121.27127,24.9693,121.28773,25.0413
2,822-1128,552.0,121.27032,24.9533,121.28773,25.0413
3,913-1127,546.0,121.28484,24.9002,121.28919,25.0378
4,903-1127,515.0,121.07294,24.9101,121.28919,25.0378
...,...,...,...,...,...,...
218563,479-608,1.0,121.51786,24.9681,121.29374,24.9880
218564,479-607,1.0,121.51786,24.9681,121.30057,24.9905
218565,479-606,1.0,121.51786,24.9681,121.30098,24.9880
218566,479-605,1.0,121.51786,24.9681,121.32130,24.9774


In [38]:
def get_OD_line_shp(df, o_col, d_col, o_x_col, o_y_col, d_x_col, d_y_col, count_col, date_col, how='countd', combine=True):
    '''
    Parameters:
    df (dataframe) : 要計算的表格，例如信令資料、票證資料統計圖表
    o_col (str) : 起點的名稱 / 站序 / 可判別的欄位
    d_col (str) : 迄點的名稱 / 站序 / 可判別的欄位
    o_x_col (str) : 起點的經度 (wgs84)
    o_y_col (str) : 起點的緯度 (wgs84)
    d_x_col (str) : 迄點的經度 (wgs84)
    d_y_col (str) : 迄點的緯度 (wgs84)
    count_col (str) : 需要統計依據的量值
    combine (Boolean) : True為去定義是否要把同一對起訖端點的量合併成同一條統計量，False則僅顯示OD
    how (str) : 填入'sum'或'mean'
    '''
    import numpy as np
    import pandas as pd

    # 若不合併雙向OD，直接計算
    if not combine: 
        agg_funcs = {o_x_col: 'mean', o_y_col: 'mean', d_x_col: 'mean', d_y_col: 'mean'}
        agg_funcs[count_col] = 'sum' if how != 'countd' else 'sum'
        
        # 聚合處理
        countdf = df.groupby([o_col, d_col]).agg(agg_funcs).reset_index()

        if how == 'countd':  # 特殊處理countd的情況
            countdf[count_col] /= countdf[date_col].nunique()
            countdf.drop(columns=date_col, inplace=True)

        return countdf.sort_values(count_col, ascending=False).reset_index(drop=True)

    # 若合併雙向OD，處理經緯度並計算
    place = pd.concat([
        df[[o_col, o_x_col, o_y_col]].rename(columns={o_col: 'PlaceName', o_x_col: 'PlaceLng', o_y_col: 'PlaceLat'}),
        df[[d_col, d_x_col, d_y_col]].rename(columns={d_col: 'PlaceName', d_x_col: 'PlaceLng', d_y_col: 'PlaceLat'})
    ]).drop_duplicates(subset=['PlaceName']).reset_index(drop=True).reset_index(names='PlaceID')

    # 合併資料
    df_copy = pd.merge(df, place[['PlaceID', 'PlaceName']], left_on=o_col, right_on='PlaceName', how='left') \
               .merge(place[['PlaceID', 'PlaceName']], left_on=d_col, right_on='PlaceName', how='left', suffixes=['_o', '_d'])

    # 計算Pair欄位
    df_copy['Pair'] = df_copy.apply(lambda x: f"{min(x['PlaceID_o'], x['PlaceID_d'])}-{max(x['PlaceID_o'], x['PlaceID_d'])}", axis=1)

    # 根據how進行分組統計
    agg_funcs = {count_col: 'sum'} if how != 'countd' else {count_col: 'sum', date_col: 'nunique'}
    countdf = df_copy.groupby(['Pair'], as_index=False).agg(agg_funcs)

    if how == 'countd':
        countdf[count_col] /= countdf[date_col].nunique()
        countdf.drop(columns=date_col, inplace=True)

    # 合併經緯度
    place_map = place.rename(columns={'PlaceID': 'PlaceID1'})
    countdf = countdf.merge(place_map, left_on='PlaceID1', right_on='PlaceID', how='left') \
                     .merge(place_map.rename(columns={'PlaceID': 'PlaceID2'}), left_on='PlaceID2', right_on='PlaceID', how='left', suffixes=['_o', '_d'])

    # 返回結果
    return countdf[['Pair', count_col, 'PlaceLng_o', 'PlaceLat_o', 'PlaceLng_d', 'PlaceLat_d']] \
           .rename(columns={'PlaceLng_o': o_x_col, 'PlaceLat_o': o_y_col, 'PlaceLng_d': d_x_col, 'PlaceLat_d': d_y_col}) \
           .sort_values(count_col, ascending=False).reset_index(drop=True)

# 呼叫函式
get_OD_line_shp(cvp, o_col =o_col , d_col = d_col , o_x_col = o_x_col, o_y_col = o_y_col , d_x_col = d_x_col , d_y_col = d_y_col , count_col = count_col, date_col = date_col, how = 'countd' ,combine = False )

KeyError: 'date'